#**Logistic Regression**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk

: 

In [2]:
data = pd.read_csv('/content/cyberbullying_tweets.csv')
data.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [3]:
!pip install textblob
!pip install scikit-learn

In [4]:
from textblob import TextBlob

In [5]:
pip install tweet-preprocessor

In [10]:
import preprocessor as p
import string
from gensim.parsing.preprocessing import remove_stopwords
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment>0:
      s=1
    elif sentiment<0:
      s=-1
    else:
      s=0
    return s

data['sentiment'] = data['tweet_text'].apply(get_sentiment)

In [7]:
print(data)

                                              tweet_text cyberbullying_type  \
0      In other words #katandandre, your food was cra...  not_cyberbullying   
1      Why is #aussietv so white? #MKR #theblock #ImA...  not_cyberbullying   
2      @XochitlSuckkks a classy whore? Or more red ve...  not_cyberbullying   
3      @Jason_Gio meh. :P  thanks for the heads up, b...  not_cyberbullying   
4      @RudhoeEnglish This is an ISIS account pretend...  not_cyberbullying   
...                                                  ...                ...   
47687  Black ppl aren't expected to do anything, depe...          ethnicity   
47688  Turner did not withhold his disappointment. Tu...          ethnicity   
47689  I swear to God. This dumb nigger bitch. I have...          ethnicity   
47690  Yea fuck you RT @therealexel: IF YOURE A NIGGE...          ethnicity   
47691  Bro. U gotta chill RT @CHILLShrammy: Dog FUCK ...          ethnicity   

       sentiment  
0             -1  
1            

In [8]:
def preprocess_tweet(text):
    text = p.clean(text)
    text = remove_stopwords(text)
    text= text.translate(str.maketrans('', '',string.punctuation))
    text.lower()
    lem = WordNetLemmatizer()
    text=lem.lemmatize(text)
    return text

In [15]:
data['tweet_text']=data['tweet_text'].apply(preprocess_tweet)
X=data['tweet_text']
y=data['cyberbullying_type']
# X = data.iloc[:, [0, 2]]  # Selecting columns at index 0 and 2
# y = data.iloc[:, 1]  # Selecting column at index 1

In [16]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [17]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # You can adjust the max_features
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [18]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [19]:
val_predictions = model.predict(X_val_tfidf)

In [20]:
accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy: {accuracy:.2f}")
print(classification_report(y_val, val_predictions))

Validation Accuracy: 0.81
                     precision    recall  f1-score   support

                age       0.95      0.97      0.96      1173
          ethnicity       0.97      0.96      0.96      1219
             gender       0.90      0.81      0.85      1182
  not_cyberbullying       0.55      0.55      0.55      1223
other_cyberbullying       0.57      0.62      0.60      1182
           religion       0.93      0.93      0.93      1175

           accuracy                           0.81      7154
          macro avg       0.81      0.81      0.81      7154
       weighted avg       0.81      0.81      0.81      7154



In [ ]:
import joblib
joblib.dump(model,'model_joblib')

In [22]:
joblib.dump(tfidf_vectorizer,'tfidf_joblib')

['tfidf_joblib']

In [ ]:
mj=joblib.load('model_joblib')

In [ ]:
mj.predict(tfidf_vectorizer.transform(["How are you?"]))

array(['not_cyberbullying'], dtype=object)

In [ ]:
test_predictions = model.predict(X_test_tfidf)

In [ ]:
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Test Accuracy: {test_accuracy:.2f}")
print(classification_report(y_test, test_predictions))

Test Accuracy: 0.80
                     precision    recall  f1-score   support

                age       0.95      0.96      0.95      1176
          ethnicity       0.98      0.96      0.97      1199
             gender       0.89      0.79      0.83      1161
  not_cyberbullying       0.55      0.56      0.55      1243
other_cyberbullying       0.56      0.62      0.59      1209
           religion       0.93      0.94      0.93      1166

           accuracy                           0.80      7154
          macro avg       0.81      0.80      0.80      7154
       weighted avg       0.81      0.80      0.80      7154



In [ ]:
input="Hello How are you?"
input=[input]
X_new_tfidf = tfidf_vectorizer.transform(input)
# Make predictions on the new data
predictions = model.predict(X_new_tfidf)
print(predictions)

['not_cyberbullying']


In [ ]:
input="You fuck off"
input=[input]
X_new_tfidf = tfidf_vectorizer.transform(input)
# Make predictions on the new data
predictions = model.predict(X_new_tfidf)
print(predictions)

['ethnicity']


In [ ]:
input="You shall be kicked on ass"
input=[input]
X_new_tfidf = tfidf_vectorizer.transform(input)
# Make predictions on the new data
predictions = model.predict(X_new_tfidf)
print(predictions)

['other_cyberbullying']


In [ ]:
input="Your religion is very bad"
input=[input]
X_new_tfidf = tfidf_vectorizer.transform(input)
# Make predictions on the new data
predictions = model.predict(X_new_tfidf)
print(predictions)

['religion']
